In [ ]:
#Import Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import shutil
import random

Experiment with: Median filter, Histogram Equalization
RGB
10 Epochs
Softmax activation function
spatse categorical cross entropy

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Navigate to your folder (replace with your actual folder path)
folder_path = '/content/drive/My Drive/split_dataset'

# List files in the folder
os.listdir(folder_path)

['images', 'split_dataset', 'labels', '__MACOSX']

In [ ]:
def load_image_and_label(image_path, label_path, image_size=(224, 224)):
    image_path = image_path.numpy().decode('utf-8')
    label_path = label_path.numpy().decode('utf-8')

    img = cv2.imread(image_path)
    image_size_int = (int(image_size[0]), int(image_size[1]))
    img = cv2.resize(img, image_size_int)

    # Median filter for noise reduction
    img = cv2.medianBlur(img, 5)

    # Normalize image
    img = img.astype(np.float32) / 255.0

    # High-pass filter for enhancement on grayscale version
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_gray = cv2.resize(img_gray, image_size_int)
    kernel = np.array([[-1, -1, -1],
                       [-1,  8, -1],
                       [-1, -1, -1]])
    high_pass = cv2.filter2D(img_gray, -1, kernel)

    # Load label and create one-hot vector
    with open(label_path, 'r') as file:
        line = file.readline().strip()
        class_id = int(line.split()[0]) if line else 0

    label_vector = np.zeros(num_classes, dtype=np.float32)
    label_vector[class_id] = 1.0

    return img, high_pass.astype(np.float32) / 255.0, label_vector

In [ ]:
def tf_load_image_and_label(image_path, label_path, image_size=(224, 224)):
    img, img_gray, label = tf.py_function(
        func=load_image_and_label,
        inp=[image_path, label_path, image_size],
        Tout=[tf.float32, tf.float32, tf.int32]
    )
    img.set_shape((image_size[0], image_size[1], 3))
    img_gray.set_shape((image_size[0], image_size[1]))
    label.set_shape(())

    return img, label

In [ ]:
import os
import tensorflow as tf

def create_dataset(image_dir, label_dir, image_size=(224, 224), batch_size=32, dataset_type='train', show_warnings=True):
    sub_image_dir = os.path.join(image_dir, dataset_type)
    sub_label_dir = os.path.join(label_dir, dataset_type)

    # Get a list of image files
    image_paths = [f for f in os.listdir(sub_image_dir) if f.endswith(('.jpg', '.png'))]

    image_file_paths = []
    label_file_paths = []

    # Check for corresponding label file for each image
    for image_name in image_paths:
        base_name = os.path.splitext(image_name)[0]
        label_name = base_name + '.txt'
        label_path = os.path.join(sub_label_dir, label_name)

        if os.path.exists(label_path):
            image_file_paths.append(os.path.join(sub_image_dir, image_name))
            label_file_paths.append(label_path)
        else:
            if show_warnings:
                print(f"Warning: Missing label file for image: {image_name}")

    if not image_file_paths:
        if show_warnings:
            print(f"Warning: No valid image-label pairs found in {sub_image_dir}")
        return tf.data.Dataset.from_tensor_slices(())

    # Create dataset and map preprocessing
    dataset = tf.data.Dataset.from_tensor_slices((image_file_paths, label_file_paths))
    dataset = dataset.map(
        lambda img_path, lbl_path: tf_load_image_and_label(img_path, lbl_path, image_size),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Shuffle if training
    if dataset_type == 'train':
        dataset = dataset.shuffle(buffer_size=len(image_file_paths))

    # Batch and prefetch
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


In [ ]:
# Number of output classes (adjust based on your dataset)
num_classes = 2  # <-- change this if you have a different number of classes

# Define the CNN model
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Define paths to your main image and label directories
image_dir = '/content/drive/My Drive/split_dataset/images'
label_dir = '/content/drive/My Drive/split_dataset/labels'

# Create the training, validation, and test datasets
train_dataset = create_dataset(image_dir, label_dir, image_size=(224, 224), batch_size=32, dataset_type='train')
val_dataset = create_dataset(image_dir, label_dir, image_size=(224, 224), batch_size=32, dataset_type='val')
test_dataset = create_dataset(image_dir, label_dir, image_size=(224, 224), batch_size=32, dataset_type='test')

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Epoch 1/10


In [ ]:
# Access loss and accuracy history
train_loss = history.history['loss']  # List of training loss values per epoch
train_accuracy = history.history['accuracy']  # List of training accuracy values per epoch
val_loss = history.history['val_loss']  # List of validation loss values per epoch
val_accuracy = history.history['val_accuracy']  # List of validation accuracy values per epoch

# Print the final values of loss and accuracy for the last epoch
print(f"Final Training Loss: {train_loss[-1]}")
print(f"Final Training Accuracy: {train_accuracy[-1]}")
print(f"Final Validation Loss: {val_loss[-1]}")
print(f"Final Validation Accuracy: {val_accuracy[-1]}")

In [ ]:
import matplotlib.pyplot as plt

# Plot Training & Validation Loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss during Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot Training & Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Train Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Accuracy during Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")